In [1]:
import math
import numpy as np
import pandas as pd
from collections import Counter
from PyImpetus import PPIMBC
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import time

In [32]:
#original_ais_df=pd.read_pickle('AIS_UNACORN_Seatracks.pkl')
original_nids_df=pd.read_csv('/storage/users/aswin1906/projects/ai/Globalhelpforall/network-anomaly-detection/data/unsw/Train_Test_Network.csv')
original_nids_df.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [3]:
original_nids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461043 entries, 0 to 461042
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ts                      461043 non-null  int64  
 1   src_ip                  461043 non-null  object 
 2   src_port                461043 non-null  int64  
 3   dst_ip                  461043 non-null  object 
 4   dst_port                461043 non-null  int64  
 5   proto                   461043 non-null  object 
 6   service                 461043 non-null  object 
 7   duration                461043 non-null  float64
 8   src_bytes               461043 non-null  int64  
 9   dst_bytes               461043 non-null  int64  
 10  conn_state              461043 non-null  object 
 11  missed_bytes            461043 non-null  int64  
 12  src_pkts                461043 non-null  int64  
 13  src_ip_bytes            461043 non-null  int64  
 14  dst_pkts            

In [33]:
original_nids_df.select_dtypes(['object'])

,src_ip,dst_ip,proto,service,conn_state,dns_query,dns_AA,dns_RD,dns_RA,dns_rejected,...,http_method,http_uri,http_version,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,type
0,3.122.49.24,192.168.1.152,tcp,-,OTH,-,-,-,-,-,...,-,-,-,-,-,-,bad_TCP_checksum,-,F,normal
1,192.168.1.79,192.168.1.255,udp,-,S0,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,normal
2,192.168.1.152,192.168.1.152,tcp,-,OTH,-,-,-,-,-,...,-,-,-,-,-,-,bad_TCP_checksum,-,F,normal
3,192.168.1.152,192.168.1.152,tcp,-,OTH,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,normal
4,192.168.1.152,192.168.1.190,udp,dns,SHR,-,-,-,-,-,...,-,-,-,-,-,-,bad_UDP_checksum,-,F,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,192.168.1.32,176.28.50.165,tcp,-,S0,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,xss
461039,192.168.1.33,34.230.157.88,tcp,-,OTH,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,ransomware
461040,192.168.1.37,192.168.1.193,tcp,-,OTH,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,backdoor
461041,192.168.1.31,104.98.29.100,tcp,-,SF,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,mitm


In [34]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# data= original_ais_df_cp.drop(['type'], axis=1)

In [35]:
categoricalColumnsInTheDataSet = original_nids_df.select_dtypes(['object'])

In [36]:
categoricalColumnNames = categoricalColumnsInTheDataSet.columns.values

In [37]:
original_nids_df

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,1556340862,192.168.1.32,33108,176.28.50.165,80,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,1,xss
461039,1556423390,192.168.1.33,37242,34.230.157.88,443,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,1,ransomware
461040,1556436603,192.168.1.37,4444,192.168.1.193,49178,tcp,-,290.371539,101568,2592,...,0,0,-,-,-,-,-,-,1,backdoor
461041,1556540442,192.168.1.31,60816,104.98.29.100,443,tcp,-,23.190902,32,31,...,0,0,-,-,-,-,-,-,1,mitm


In [38]:
dataSetForFeatureEncoding=original_nids_df

In [39]:
dataSetForFeatureEncoding.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [40]:
labelEncoder = LabelEncoder() 
for feature in categoricalColumnNames:
    dataSetForFeatureEncoding[feature] = labelEncoder.fit_transform(dataSetForFeatureEncoding[feature]) 

In [21]:
dataSetForFeatureEncoding.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,8047,1883,1511,52976,1,0,80549.530260,1762852,41933215,...,0,0,0,0,0,5,0,1,0,normal
1,1554198358,4614,47260,1522,15600,2,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,normal
2,1554198359,4590,1880,1511,51782,1,0,0.000000,0,0,...,0,0,0,0,0,5,0,1,0,normal
3,1554198359,4590,34296,1511,10502,1,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,normal
4,1554198362,4590,46608,1516,53,2,3,0.000549,0,298,...,0,0,0,0,0,6,0,1,0,normal


In [22]:
data= dataSetForFeatureEncoding.drop(['type'], axis=1)

In [23]:
data

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,1554198358,8047,1883,1511,52976,1,0,80549.530260,1762852,41933215,...,0,0,0,0,0,0,5,0,1,0
1,1554198358,4614,47260,1522,15600,2,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1554198359,4590,1880,1511,51782,1,0,0.000000,0,0,...,0,0,0,0,0,0,5,0,1,0
3,1554198359,4590,34296,1511,10502,1,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1554198362,4590,46608,1516,53,2,3,0.000549,0,298,...,0,0,0,0,0,0,6,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,1556340862,4604,33108,1276,80,1,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,1
461039,1556423390,4605,37242,4154,443,1,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,1
461040,1556436603,4608,4444,1518,49178,1,0,290.371539,101568,2592,...,0,0,0,0,0,0,0,0,0,1
461041,1556540442,4603,60816,104,443,1,0,23.190902,32,31,...,0,0,0,0,0,0,0,0,0,1


In [43]:
data, Y = dataSetForFeatureEncoding.drop(['type'], axis=1), dataSetForFeatureEncoding['type'].values

In [ ]:
# We want to time our algorithm
start = time.time()
# Use KFold for understanding the performance of PyImpetus
kfold = KFold(n_splits=5, random_state=27, shuffle=True)
# This will hold all the accuracy scores
scores = list()
# Perform CV
for train, test in kfold.split(data):
    # Split data into train and test based on folds
    x_train, x_test = data.iloc[train], data.iloc[test]
    y_train, y_test = Y[train], Y[test]

    # Create a PyImpetus classification object and initialize with required parameters
    # NOTE: To achieve fast selection, set cv=0 for disabling the use of any internal cross-validation
    model = PPIMBC(LogisticRegression(random_state=27, max_iter=1000, class_weight="balanced"), cv=0, num_simul=20, simul_type=0, simul_size=0.2, sig_test_type="non-parametric", random_state=27, verbose=2, p_val_thresh=0.05)
    # Fit this above object on the train part and transform the train dataset into selected feature subset
    # NOTE: x_train has to be a dataframe and y_train has to be a numpy array
    x_train = model.fit_transform(x_train, y_train)
    # Transform the test set as well
    # NOTE: x_test has to be a dataframe
    x_test = model.transform(x_test)
    # Check out the features selected
    print("Markov Blanket: ", model.MB)
    # Check out the scores of each feature. The scores are in order of the selected feature list
    # NOTE: You can use these scores ina feature selection ensemble
    print("Feature importance: ", model.feat_imp_scores)
    # Plot the feature importance scores
    model.feature_importance()
    # Convert the data into numpy arrays
    x_train, x_test = x_train.values, x_test.values
    
    model = DecisionTreeClassifier(random_state=27)
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    score = accuracy_score(y_test, preds)
    scores.append(score)
    print("Score: ", score)
# Compute average score
print("\n\nAverage Accuracy: ", sum(scores)/len(scores))
# Finally, check out the total time taken
end = time.time()
print("\n\nTotal Time Required (in seconds): ", end-start)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 out of  44 | elapsed:  1.1min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  37 out of  44 | elapsed:  5.0min remaining:   56.8s
[Parallel(n_jobs=-1)]: Done  44 out of  44 | elapsed:  9.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  20 | elapsed:   11.3s remaining:   34.0s
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:   20.1s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   24.1s finished
/home/users/aswin1906/.local/lib/python3.8/site-packages/scipy/stats/morestats.py:2967: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out 